This is my code understanding. I will do step by step. Lets do it!!!

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

/media/e-soc-student/DISK2/GR/GR2_Recommendation/faiss_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# This code does load model (huggingdace)

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading LLaMA 3-8B...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16, # I think this is memmory efficient, less VRAM usage!!
    device_map="auto" #This line is to make sure model is loaded on GPU (however, in Manami's computer, it becomes CPU cuz simply, the my gpu cannot handle it. )
)


▶ Loading LLaMA 3-8B...


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.28it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:
# This is the generation pipeline 

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [4]:
# Now, i make sure the llama works good 

paragraph = "Explain why knowledge graphs are useful for artificial intelligence research."

In [5]:
response = generator(paragraph, max_new_tokens=150, temperature=0.0, do_sample=False)

# Max new tokens are set to 150 to limit the LLM's answer (but the token is in addition to the input)
# temperature tells what kind of output. For example, 0.0 tells always same output for the same input but 1.0 has balanced randomness. so 0.0 tells conssistent and accurate answers
# Sample also tells the randomness. so if i set sample = True, then temerature tells how much randomness i want. 

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [6]:
print("\n▶ LLaMA Output:\n")
print(response[0]["generated_text"])
# response 0 tells the first item in the list (list of dictionary)


▶ LLaMA Output:

Explain why knowledge graphs are useful for artificial intelligence research. Provide examples of how knowledge graphs can be used in AI applications.
Knowledge graphs are a type of data structure that represents entities and their relationships in a graph-like structure. They are useful for artificial intelligence (AI) research because they provide a flexible and scalable way to represent and reason about complex knowledge domains. Here are some reasons why knowledge graphs are useful for AI research:

1. **Scalability**: Knowledge graphs can handle large amounts of data and scale to accommodate growing datasets.
2. **Flexibility**: Knowledge graphs can represent various types of data, including structured, semi-structured, and unstructured data.
3. **Reasoning**: Knowledge graphs enable reasoning and inference capabilities, allowing AI systems to draw conclusions and make predictions based on the relationships between


Next is how to understand user paragraph.

To do that: <br>
Step 1: Prompt LLaMA to extract concepts (prompt tuning)<br>
Step2: Estimate importance score (if possible, but can LLaMA do? )


Prompt engineering explanation is here:
https://www.promptingguide.ai/techniques/cot

In [7]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import re, json

# 1) Define your LLaMA pipeline & extract_concepts() in one cell:
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                    device_map="auto", torch_dtype="auto")
llm       = pipeline("text-generation", model=model, tokenizer=tokenizer)


PROMPT = """
You are an academic assistant. Extract only the most important research concepts from the paragraph as a JSON list:
<TEXT>{paragraph}</TEXT>
Expected format: {{"concepts":["concept1","concept2","..."]}}
"""

# this is the Zero-shot prompt engineering. 

def extract_concepts(paragraph):
    formatted = PROMPT.format(paragraph=paragraph.strip())
    output = llm(formatted, max_new_tokens=256, temperature=0.0, do_sample=False)[0]["generated_text"]

    # Extract valid JSON from output
    # This line try to search all strings like this (json-looking substrings) and (re is search for patterns)
    # The json file is made like this {"concepts": ["knowledge graphs", "AI systems", "data integration"]}
    for match in re.findall(r"\{[^{}]+\}", output, re.S):
        try:
            data = json.loads(match)
            if "concepts" in data:
                return data["concepts"]
        except Exception:s
            continue
    raise ValueError("Could not parse LLaMA output:\n" + output)

# # Example usage
# if __name__ == "__main__":
#     paragraph = """
#     Therefore, knowledge graphs have seized great opportunities by improving the quality of AI systems 
#     and being applied to various areas. However, the research on knowledge graphs still faces significant 
#     technical challenges. For example, there are major limitations in the current technologies for acquiring 
#     knowledge from multiple sources and integrating them into a typical knowledge graph. Thus, knowledge graphs 
#     provide great opportunities in modern society. However, there are technical challenges in their development. 
#     Consequently, it is necessary to analyze the knowledge graphs with respect to their opportunities and challenges 
#     to develop a better understanding of the knowledge graphs.
#     """

#     concepts = extract_concepts(paragraph)
#     print("\n▶ Extracted Concepts:")
#     for c in concepts:
#         print("-", c)
#     from pprint import pprint

# # after you get `concepts`:
#     pprint(concepts, width=100)



IndentationError: unexpected indent (397165676.py, line 33)

In [ ]:
# 2) In a *separate* cell, call and print:
from pprint import pprint

paragraph = """
    Therefore, knowledge graphs have seized great opportunities by improving the quality of AI systems 
    and being applied to various areas. However, the research on knowledge graphs still faces significant 
    technical challenges. For example, there are major limitations in the current technologies for acquiring 
    knowledge from multiple sources and integrating them into a typical knowledge graph. Thus, knowledge graphs 
    provide great opportunities in modern society. However, there are technical challenges in their development. 
    Consequently, it is necessary to analyze the knowledge graphs with respect to their opportunities and challenges 
    to develop a better understanding of the knowledge graphs.
    """
concepts = extract_concepts(paragraph)

print("▶ Extracted Concepts:")
for c in concepts:
    print(" -", c)

print("\n▶ Full list via pprint:")
pprint(concepts, width=80)
